In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [2]:
np.random.seed(1)

In [3]:
df = pd.read_csv("hcc.csv")
df.head()

,1.Gen,2.Sym,3.Alc,4.HepB,5.HepB,6.HepB,7.HepC,8.Cir,9.End,10.Smo,...,Unnamed: 246,Unnamed: 247,Unnamed: 248,Unnamed: 249,Unnamed: 250,Unnamed: 251,Unnamed: 252,Unnamed: 253,Unnamed: 254,Unnamed: 255
0,1,0,1,0,0,0,0,1,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,?,0,0,0,0,1,1,?,?,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,0,1,1,0,1,0,1,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1,1,0,0,0,0,1,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,1,1,1,0,1,0,1,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165 entries, 0 to 164
Columns: 256 entries, 1.Gen to Unnamed: 255
dtypes: float64(206), int64(6), object(44)
memory usage: 330.1+ KB


## Data Cleaning

In [5]:
target = 'Class'

### Replace empty or errors by np.nan

In [6]:
# replace empty/erros by np.nan
df = df.replace(r'^\s+$', np.nan, regex=True)
df = df.replace('\t','',regex=True)
df = df.replace(' ','',regex=True)
df = df.replace('\?',np.nan,regex=True)
df = df.replace('\<',np.nan,regex=True)
df = df.replace('#NULL!',np.nan,regex=True)
#df = df.replace('99',np.nan,regex=True)
#df = df.replace(99,np.nan,regex=True)
#df

### Remove outliers

In [7]:
# Drop the column outliers as it is unrelated to the dependent variable
outliers = []
df = df.drop(outliers,axis=1)
df.head()

,1.Gen,2.Sym,3.Alc,4.HepB,5.HepB,6.HepB,7.HepC,8.Cir,9.End,10.Smo,...,Unnamed: 246,Unnamed: 247,Unnamed: 248,Unnamed: 249,Unnamed: 250,Unnamed: 251,Unnamed: 252,Unnamed: 253,Unnamed: 254,Unnamed: 255
0,1,0,1,0,0,0,0,1,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,NaN,0,0,0,0,1,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,0,1,1,0,1,0,1,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1,1,0,0,0,0,1,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,1,1,1,0,1,0,1,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Remove bad columns

In [8]:
def remove_bad_columns(df,bad_column_threshold):
    # find bad columns having too many missing values
    n_null = np.array(df.isnull().sum(axis=0))
    bad_col = np.array([]).astype(int)
    for i in range(len(n_null)):
        if n_null[i] >= bad_column_threshold:
            bad_col = np.append(bad_col,i)

    #print(bad_col)
    print('number of bad columns:',len(bad_col))

    # delete bad columns
    df = df.drop(df.columns[bad_col],axis=1)
    #df.info()
    return df  

In [9]:
df = remove_bad_columns(df,20)
df.info()

number of bad columns: 220
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165 entries, 0 to 164
Data columns (total 36 columns):
1.Gen           165 non-null int64
2.Sym           147 non-null object
3.Alc           165 non-null int64
4.HepB          148 non-null object
7.HepC          156 non-null object
8.Cir           165 non-null int64
11.Dia          162 non-null object
12.Obe          155 non-null object
14.Art          162 non-null object
15.CRen         163 non-null object
16.HIV          151 non-null object
19.Spl          150 non-null object
20.PHyp         154 non-null object
21.Thr          162 non-null object
22.LMet         161 non-null object
23.Rad          163 non-null object
24.Agedia       165 non-null int64
27.Sta          165 non-null int64
28.Encdeg       164 non-null object
29.Ascdeg       163 non-null object
30.IntNorRat    161 non-null object
31.Alp          157 non-null object
32.Hae          162 non-null object
33.MCorVol      162 non-null object
34.Leu   

### Remove bad rows

In [10]:
 # Find rows where target is missing
def find_missing_target_rows(df,target):
    # find rows where target is missing
    missing_row = df[target].isnull()
    print('Number of rows where target are missing:')
    print(sum(missing_row))

    #df = df[~missing_row]
    missing_row_indices = np.array([t for t in range(df.shape[0]) if missing_row[t]])
    
    return missing_row_indices

In [11]:
missing_target_rows = find_missing_target_rows(df,target)

Number of rows where target are missing:
0


We find bad rows which contain too many missing values, then remove them.

In [12]:
def find_bad_rows(df,bad_row_threshold):   
    # find bad rows having too many missing values
    n_null = np.array(df.isnull().sum(axis=1))
    bad_row = np.array([]).astype(int)
    for t in range(len(n_null)):
        if n_null[t] >= bad_row_threshold:
            bad_row = np.append(bad_row,t)

    #print(bad_row)
    print('number of bad rows:',len(bad_row))

    # delete bad rows
    #df = df.drop(bad_row)
    #df.info()
    return bad_row

In [13]:
bad_rows = find_bad_rows(df,10)
df.info()

number of bad rows: 3
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165 entries, 0 to 164
Data columns (total 36 columns):
1.Gen           165 non-null int64
2.Sym           147 non-null object
3.Alc           165 non-null int64
4.HepB          148 non-null object
7.HepC          156 non-null object
8.Cir           165 non-null int64
11.Dia          162 non-null object
12.Obe          155 non-null object
14.Art          162 non-null object
15.CRen         163 non-null object
16.HIV          151 non-null object
19.Spl          150 non-null object
20.PHyp         154 non-null object
21.Thr          162 non-null object
22.LMet         161 non-null object
23.Rad          163 non-null object
24.Agedia       165 non-null int64
27.Sta          165 non-null int64
28.Encdeg       164 non-null object
29.Ascdeg       163 non-null object
30.IntNorRat    161 non-null object
31.Alp          157 non-null object
32.Hae          162 non-null object
33.MCorVol      162 non-null object
34.Leu        

In [14]:
del_rows = np.union1d(missing_target_rows,bad_rows)
print('number of rows need to delete:',len(del_rows))

df = df.drop(del_rows)

number of rows need to delete: 3


### Separate target and attributes

In [15]:
dfx = df.drop(target,axis=1)
dfy = df[target]

### Find variable type

In [16]:
# number of uniques of each column (excluding NaN)
nu = np.array([len(pd.unique(dfx[col].dropna())) for col in dfx.columns])
print('number of uniques of each variable:')
print(nu)

number of uniques of each variable:
[  2   2   2   2   2   2   2   2   2   2   2   2   2   2   2   2  50   5
   3   3  87 131  71 128 105 131  41  62  93 107 139 124  46  84   6]


In [17]:
def define_variable_type(df,nu):
    i_binary = [] ; i_category = [] ; i_continuous = []
    for i in range(len(nu)):
        if nu[i] == 2: # binary 
            i_binary.append(i)
        elif nu[i] < 5: # !!!! NOTE: this is not always correct, depending on data
            i_category.append(i)
        else:
            i_continuous.append(i)

    print('i_binary:',i_binary)
    print('i_category:',i_category)   
    #i_binary, i_category, i_continuous
    
    variable_type  = np.ones(len(nu))  # binary
    variable_type[i_category] = 2   # categorical
    variable_type[i_continuous] = 3 # continuous

    return variable_type #,i_binary,i_category,i_continuous

In [18]:
variable_type = define_variable_type(dfx,nu)
print('variable type:',variable_type)

i_binary: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
i_category: [18, 19]
variable type: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 3. 3. 2. 2. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.]


### Impute missing values of attributes

In [19]:
def impute_missing(df,variable_type):
    # impute binary and categorical variables by the most frequency (in each column)
    # continuous variable by median
    # input: df: pandas data frame, variable_type: list
    # output: df2: pandas data frame
    df2 = df.copy()
    for i,col in enumerate(df.columns):
        if variable_type[i] < 3: # binary or caterogy
            df2[col] = df[col].fillna(df[col].mode().iloc[0])
        else: # continuous
            df2[col] = df[col].fillna(df[col].median())    
    return df2       

In [20]:
dfx_imputed = impute_missing(dfx,variable_type)
dfx_imputed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 162 entries, 0 to 164
Data columns (total 35 columns):
1.Gen           162 non-null int64
2.Sym           162 non-null object
3.Alc           162 non-null int64
4.HepB          162 non-null object
7.HepC          162 non-null object
8.Cir           162 non-null int64
11.Dia          162 non-null object
12.Obe          162 non-null object
14.Art          162 non-null object
15.CRen         162 non-null object
16.HIV          162 non-null object
19.Spl          162 non-null object
20.PHyp         162 non-null object
21.Thr          162 non-null object
22.LMet         162 non-null object
23.Rad          162 non-null object
24.Agedia       162 non-null int64
27.Sta          162 non-null int64
28.Encdeg       162 non-null object
29.Ascdeg       162 non-null object
30.IntNorRat    162 non-null object
31.Alp          162 non-null object
32.Hae          162 non-null object
33.MCorVol      162 non-null object
34.Leu          162 non-null object


## Data Processing

### Attributes

In [21]:
def convert_binary_and_category(x,variable_type):
    """
    convert binary to +-1, category to one hot; remain continuous.
    """    
    onehot_encoder = OneHotEncoder(sparse=False,categories='auto')

    # create 2 initial columns
    x_new = np.zeros((x.shape[0],2))

    for i,i_type in enumerate(variable_type):
        if i_type == 1: # binary
            unique_value = np.unique(x[:,i])
            x1 = np.array([-1. if value == unique_value[0] else 1. for value in x[:,i]])        
            x_new = np.hstack((x_new,x1[:,np.newaxis]))

        elif i_type == 2: # category
            x1 = onehot_encoder.fit_transform(x[:,i].reshape(-1,1))
            x_new = np.hstack((x_new,x1))
            
        else: # continuous      
            x_new = np.hstack((x_new,x[:,i][:,np.newaxis]))      

    # drop the 2 initial column
    x_new = x_new[:,2:]
    
    return x_new.astype(float)

In [22]:
# convert x
x = np.array(dfx_imputed)
x_new = convert_binary_and_category(x,variable_type)

print(x_new.shape)
print(x_new)

(162, 39)
[[ 1.   -1.    1.   ...  7.1   0.7   1.  ]
 [ 1.   -1.    1.   ...  7.    2.1   5.  ]
 [ 1.    1.    1.   ...  8.1   1.11  2.  ]
 ...
 [ 1.   -1.    1.   ...  7.5   1.46  5.  ]
 [ 1.   -1.    1.   ...  8.4   0.74  5.  ]
 [ 1.    1.    1.   ...  6.6   3.95  5.  ]]


### Target

In [23]:
y = np.array(dfy)
#print(np.unique(y,return_counts=True))

# convert taget to 0 and 1
y_new = y
#y_new = np.ones(y.shape[0])
#y_new[y =='No'] = 0

print(np.unique(y_new,return_counts=True))

(array([0, 1]), array([63, 99]))


In [24]:
# combine X and y and save to a file
xy_new = np.hstack((x_new,y_new[:,np.newaxis]))
np.savetxt('data_processed.dat',xy_new,fmt='%f')